# LightRAG API 端點
詳細原始文件參考
http://localhost:9621/redoc#tag/documents/operation


`-X`: 指定 HTTP 方法  
`-H`: 加入 HTTP 標頭  
> ex. `-H "Content-Type: application/json"` 用於告知伺服器此次請求的資料格式是 JSON。  
> 可多次使用 `-H` 加標頭。
> 
`-d`: 傳送請求資料，常搭配 `POST`,`PUT` 使用，當指定 `Content-Type: application/json` 時，會把內容當 JSON 傳送。  
`-v`: verbose 模式，顯示完整請求與回應過程，主要用於除錯。  
`-o <file>`: 輸出到檔案中。  



In [ ]:
# http://lightrag:9621

## documents

### Scan For New Documents  
`http://localhost:9621/documents/scan`  
啟動背景掃描，去檢查輸入目錄中是否有新的文件，若有則讀取這些文件。  

**回傳內容**  
* `status`(required): Status of the scanning operation.  
   Value: `scanning_started`
* `message`: Additional details about the scanning operation.

In [2]:
!curl -X POST "http://lightrag:9621/documents/scan"

{"status":"scanning_started","message":"Scanning process has been initiated in the background"}

### Upload To Input Dir
`http://localhost:9621/documents/upload`  
將檔案上傳到指定的目錄，再對其進行索引，以便檢索。  

**傳入參數**  
* `file`(required): 要上傳的檔案。
* `api_key_header_value`: 有些伺服器可能需要 API Key 做身份驗證。  
  
**回傳內容**  
* `status`(required): Status of the uploadding operation.  
  Enum: `success`、`duplicated`、`partial_success`、`failure`  
* `message`(required): Message describing the operation result.

舉例，取得 Token:  
```
curl -X POST "http://localhost:9621/login" \
     -H "Content-Type: application/json" \
     -d '{"username": "your_user", "password": "your_pass"}'
```

使用獲得的 Token 上傳檔案:  
```
curl -X POST "http://localhost:9621/documents/upload" \
     -H "Authorization: Bearer abcd1234" \
     -F "file=@/home/user/Downloads/prince/chapter1.txt"
```  
`-F "file=@檔案路徑"` 用於傳  multipart/form-data。

In [4]:
!curl -X POST "http://lightrag:9621/documents/upload" \
    -F "file=@./../prince_docs/little_prince_1.txt"

{"status":"success","message":"File 'little_prince_1.txt' uploaded successfully. Processing will continue in background."}

### Insert Text
`http://localhost:9621/documents/text`  
用於插入純文字進 RAG 中，並得以用於之後的檢索與回應生成。  

**傳入參數**  
* `api_key_header_value`: 有些伺服器可能需要 API Key 做身份驗證。

**body 參數**  
* `text`(required): 要上傳的檔案。
* `file_source`: 來源檔案名稱或來源描述。
  
**回傳內容**  
* `status`(required): Status of the operation.  
  Enum: `success`、`duplicated`、`partial_success`、`failure`  
* `message`(required): Message describing the operation result.

In [8]:
!curl -X POST "http://lightrag:9621/documents/text" \
  -d '{ "file_source": "打招呼.txt", \
        "text": "中文：你好\n 英文：hello" \
      }'

{"detail":[{"type":"model_attributes_type","loc":["body"],"msg":"Input should be a valid dictionary or object to extract fields from","input":"{ \"file_source\": \"打招呼.txt\",          \"text\": \"中文：你好\\n 英文：hello\"        }"}]}

若沒有設定 `Content-Type` 則後端會將其視為純文字或無法解析。  
舉例有以下格式：  
* `application/json`: 解析成 JSON 物件  
* `multipart/form-data`: 解析成檔案上傳格式  
* `application/x-www-form-urlencoded`: 解析成表單  

In [10]:
!curl -X POST "http://lightrag:9621/documents/text" \
  -H "Content-Type: application/json" \
  -d '{ "file_source": "打招呼.txt", \
        "text": "中文：你好\n 英文：hello\n 日文：こんにちは" \
      }'

{"status":"success","message":"Text successfully received. Processing will continue in background."}

In [ ]:
!curl -X POST "http://lightrag:9621/documents/text" \
  -H "Content-Type: application/json" \
  -d '{
    "file_source": "相對論科普_1.txt", \
    "text": "# 相對論是什麼？一場顛覆常識的物理革命\n\n你知道嗎？在我們的直覺裡，時間是固定的、空間是絕對的、光是有速度的最快東西——但這些都被愛因斯坦的相對論徹底顛覆了！\n\n---\n\n## 相對論的兩個版本\n\n1. **狹義相對論**（1905 年）  \n   適用於不考慮重力的情況，主要探討運動、時間、空間與光速的關係。\n\n2. **廣義相對論**（1915 年）  \n   是狹義相對論的延伸，加入了重力，提出重力其實是時空的彎曲。\n\n---\n\n## 狹義相對論的兩大核心原理\n\n1. **相對性原理**  \n   所有做等速直線運動的觀察者，觀察到的物理定律都是一樣的，沒有絕對靜止的參考系。\n\n2. **光速不變原理**  \n   光在真空中的速度（約為每秒 30 萬公里）對所有觀察者來說都是相同的，無論觀察者自身是否在運動。\n\n---\n\n## 狹義相對論的驚人預測：時間變慢\n\n當一個物體以接近光速運動時，時間會對它變得更慢，這稱為「時間膨脹」（Time Dilation）。\n\n### 範例  \n如果一位太空人乘坐接近光速的飛船旅行一年，當他返回地球時，地球上可能已經過了幾十年。這個效應已經在高速粒子的實驗中被證實。\n\n---\n\n## 廣義相對論：重力是時空彎曲\n\n愛因斯坦認為，質量會讓時空彎曲，而其他物體只是沿著這個彎曲的時空自由移動。\n\n### 比喻  \n想像把一個重球放在彈簧床上，床面會下陷。如果再放上一個小球，它會向重球方向滾動，這不是因為有「力」在拉它，而是因為它沿著變形的表面運動。\n\n---\n\n## 相對論的現代應用\n\n- **GPS 定位系統**  \n  衛星上的原子鐘會因為運動速度快與重力弱而走得不一樣，必須考慮狹義與廣義相對論的修正。\n\n- **粒子加速器**  \n  例如 CERN 的大型強子對撞機，粒子被加速至接近光速，相對論效應必須納入計算。\n\n- **黑洞與引力波**  \n  廣義相對論預測了黑洞與引力波的存在，這些現象已經被天文觀測所證實。\n\n---\n\n## 結語：改變我們對宇宙的理解\n\n愛因斯坦的相對論改變了我們對時間、空間與重力的看法，揭示了宇宙不再是靜止的背景舞台，而是一個充滿變化與彎曲的四維結構。這是一場真正的物理學革命，也是一場思想上的突破。\n" \
  }'

### Insert Texts
`http://localhost:9621/documents/texts`  
用於一次性插入多筆純文字進 RAG 中，並得以用於之後的檢索與回應生成。  

**body 參數**  
* `texts`(required): 要上傳的文字，至少一筆。格式：array of strings.
* `file_sources`: 來源檔案名稱或來源描述。格式：array of strings.  
  
**回傳內容**  
* `status`(required): Status of the operation.  
  Enum: `success`、`duplicated`、`partial_success`、`failure`  
* `message`(required): Message describing the operation result.

In [17]:
!curl -X POST "http://lightrag:9621/documents/texts" \
     -H "Content-Type: application/json" \
     -d '{ "file_sources": ["physics_gravity.txt", "fluid_mechanics.txt"],\
           "texts": [\
             "重力是物體之間由質量所產生的相互吸引力，影響行星運行與自由落體運動。",\
             "流體力學研究液體與氣體的運動與作用力，包括壓力、浮力與黏滯力等現象。"\
           ]}'

{"status":"success","message":"Text successfully received. Processing will continue in background."}

### documents
`http://localhost:9621/documents`  
用於獲取系統中所有文件的狀態。  

**可查詢的狀態：**  
* PENDING: 等待處理
* PROCESSING: 正在處理
* PROCESSED: 已完成處理
* FAILED: 處理失敗

In [ ]:
!curl -X GET "http://lightrag:9621/documents" \
     -H "Content-Type: application/json"

In [1]:
!curl -X GET "http://lightrag:9621/documents" \
     -H "Content-Type: application/json" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 32374  100 32374    0     0  8678k      0 --:--:-- --:--:-- --:--:-- 10.2M
{
  "statuses": {
    "processed": [
      {
        "id": "doc-bac851bbc1cece8fa6de488b468cd338",
        "content_summary": "【小王子】－〔第十三章〕\n\n第四顆行星是實業家的行星。這位先生是這樣忙碌，以致於小王子到達的時候，他連頭也不肯抬起來看一看。\n\n「你好！」來者對他說：「你的香煙熄了。」\n\n「三加二是五，五加七、十二，十二加三、十五。你好。十五加七、二十二。二十二加六、二十八。我沒時間點火。二十六加五、三十一。呃！這樣一共是五億零一百六十二萬二千七百三十一。」\n\n「五億的什麼？」\n\n「五億一百萬的什麼？」小王子再問他。在他一生中，一旦問了問題就從來不放棄。實業家抬起了頭：\n\n「我從住在這個行星五十四年以來，只不過...",
        "content_length": 1422,
        "status": "processed",
        "created_at": "2025-08-05T06:37:33.402742+00:00",
        "updated_at": "2025-08-05T08:20:55.364513+00:00",
        "chunks_count": 2,
        "error": null,
        "metadata": {},
        "file_path": "little_prince_13.txt"
      },
      {
        "id": "doc-8c0c8eddcd4a7848b410ede1235b493e",


### Get Pipeline Status
`http://localhost:9621/documents/pipeline_status`  
取得文件索引管道的當前狀態。  

| 欄位名稱               | 類型      | 說明                                       | 預設值                 |
| ------------------ | ------- | ---------------------------------------- | ------------------- |
| `autoscanned`      | boolean | 是否啟用自動掃描功能                               | `false`             |
| `busy`             | boolean | 管線目前是否忙碌中                                | `false`             |
| `job_name`         | string  | 當前工作名稱（例：`Default Job`、`Indexing Files`） | `"Default Job"`     |
| `job_start`        | string  | 任務開始時間（ISO 格式字串）                         | `null` 或 `"string"` |
| `docs`             | integer | 總文件數量                                    | `0`                 |
| `batchs`           | integer | 批次數量                                     | `0`                 |
| `cur_batch`        | integer | 目前正在處理的批次索引                              | `0`                 |
| `request_pending`  | boolean | 是否有待處理的請求                                | `false`             |
| `latest_message`   | string  | 最新管線訊息                                   | `""`                |
| `history_messages` | array   | 歷史訊息列表                                   | `[]`                |
| `update_status`    | object  | 更新狀態資訊（例如進度、最後更新時間）                      | `{}`                |

In [3]:
!curl http://lightrag:9621/documents/pipeline_status | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 36134  100 36134    0     0  2059k      0 --:--:-- --:--:-- --:--:-- 2075k
{
  "autoscanned": false,
  "busy": false,
  "job_name": "little prince-1-11-c...[11 files]",
  "job_start": "2025-08-05T08:18:45.632053+00:00",
  "docs": 11,
  "batchs": 11,
  "cur_batch": 11,
  "request_pending": false,
  "latest_message": "Document processing pipeline completed",
  "history_messages": [
    "Processing 11 document(s)",
    "Extracting stage 1/11: little prince-1-11-checkpoint.pdf",
    "Processing d-id: doc-e6728b3f414342965449a7d4d675b099",
    "Extracting stage 2/11: little prince-12-22-6-11-checkpoint.pdf",
    "Processing d-id: doc-17d1b0d565eddc9998520f96dd29a1b3",
    "Failed to extract entities and relationships: model runner has unexpectedly stopped, this may be due to resource limitations or an internal error, check ollama s

### Delete a document and all its associated data by its ID
`http://localhost:9621/documents/delete_document`  
用於刪除指定文件及其所有關聯資料，包括：
* 文件狀態
* 文字分塊 (text chunks)
* 向量嵌入 (vector embeddings)
* 相關知識圖譜 (graph data)

刪除操作在背景中執行，以避免阻塞客戶端連線, 此操作不可逆，並會影響 pipeline 狀態。


**傳入參數**  
* `doc_ids`(required): The IDs of the documents to delete.
* `delete_file`: boolean, Whether to delete the corresponding file in the upload directory.
  
**回傳內容**  
* `status`(required): 刪除操作狀態  
    * `deletion_started` → 刪除已啟動
    * `busy" → pipeline` 正在執行其他操作
    * `not_allowed` → 當前 LLM cache 設定不允許刪除

* `message`(required): 描述此次刪除操作的訊息
* `doc_id`(required): 被刪除的文件 ID

In [3]:
!curl -X DELETE "http://lightrag:9621/documents/delete_document" \
     -H "Content-Type: application/json" \
     -d '{ "doc_ids": ["打招呼.txt"], \
           "delete_file": false \
         }'


{"status":"deletion_started","message":"Document deletion for 1 documents has been initiated. Processing will continue in background.","doc_id":"打招呼.txt"}

In [4]:
# 獲取執行刪除後的 pipeline 狀態
!curl http://lightrag:9621/documents/pipeline_status 

{"autoscanned":false,"busy":false,"job_name":"Deleting 1 Documents","job_start":"2025-08-06T07:10:14.036303","docs":1,"batchs":1,"cur_batch":1,"request_pending":false,"latest_message":"Deletion completed: 0 successful, 1 failed","history_messages":["Starting document deletion process","Deleting document 1/1: 打招呼.txt","Starting deletion process for document 打招呼.txt","Failed to delete 1/1: 打招呼.txt[] - Document 打招呼.txt not found.","Deletion completed: 0 successful, 1 failed"],"update_status":{"full_docs":[false],"text_chunks":[false],"entities":[false],"relationships":[false],"chunks":[false],"chunk_entity_relation":[false],"llm_response_cache":[false],"doc_status":[false]}}

### Clear Documents
`http://localhost:9621/documents`  
用來 清除 RAG 系統中的所有文件、實體（entities）、關係（relations）以及檔案。
它會使用底層的儲存清理方法，確保資料與輸入目錄中的檔案都被刪除，達到徹底清理的效果。  
這是不可逆操作，一旦執行所有資料都會被清空。


In [ ]:
!curl -X DELETE "http://localhost:9621/documents/clear" \
     -H "Content-Type: application/json"

### Delete Entity
`http://localhost:9621/documents/delete_entity`  
從知識圖譜中刪除指定主體與與其相連的所有關係。  

**傳入參數**  
* `entity_name`(required): 要刪除的 entity 名稱。
  
**回傳內容**  
| 欄位名稱          | 類型      | 說明                                 |
| ------------- | ------- | ---------------------------------- |
| `status`      | string  | `"success"`、`"not_found"`、`"fail"` |
| `doc_id`      | string  | 被刪除實體所屬文件的 ID                      |
| `message`     | string  | 回傳訊息，例如成功或失敗原因                     |
| `status_code` | integer | 通常為 `200`                          |
| `file_path`   | string  | 所屬文件的路徑（如 `little_prince_14.txt`）  |

In [1]:
!curl -X DELETE "http://lightrag:9621/documents/delete_entity" \
     -H "Content-Type: application/json" \
     -d '{"entity_name": "假日"}'


{"detail":"Entity '假日' not found."}

In [2]:
!curl -X DELETE "http://lightrag:9621/documents/delete_entity" \
     -H "Content-Type: application/json" \
     -d '{"entity_name": "節日"}'


{"status":"success","doc_id":"","message":"Entity '節日' and its 0 relationships have been deleted.","status_code":200,"file_path":null}

### Delete Relation
`http://localhost:9621/documents/delete_relation`  
透過指定來源實體 (source_entity) 與目標實體 (target_entity)達成刪除知識圖譜（Knowledge Graph）中兩個實體（entity）之間的關係。  

**傳入參數**  
* `source_entity`: 要刪除關係的來源實體名稱。
* `target_entity`: 要刪除關係的目標實體名稱。

**回傳內容**  
* `status`(required): 狀態：`success`、`not_found`、`fail`。
* `doc_id`(required): string, 相關的文件 ID。
* `message`(required): string, 操作結果訊息。
* `status_code`: int, HTTP 狀態碼，成功為 `200`。
* `file_path`: string, 來源檔案路徑（若有）。

In [5]:
!curl -X DELETE "http://lightrag:9621/documents/delete_relation" \
     -H "Content-Type: application/json" \
     -d '{ "source_entity": "Japanese", \
           "target_entity": "こんにちは" \
         }'


{"status":"success","doc_id":"","message":"Successfully deleted relation from 'Japanese' to 'こんにちは'","status_code":200,"file_path":null}

### Clear Cache
`http://localhost:9621/documents/clear_cache`  
用來 清除 LLM 回應快取（cache）資料, 可以針對特定模式清理，也可以一次清除全部。

`default`：代表抽取（extraction）階段的快取。  
其他模式（如 `local`, `global`, `hybrid`, `mix`, `naive`）對應不同的查詢模式。  

**參數說明**  
* `modes` (Array of strings): 要清除的快取模式。如果傳空值或不提供，則清除所有模式的快取。  
  可選值：`default`, `naive`, `local`, `global`, `hybrid`, `mix` 

In [ ]:
# 清除 "default" 與 "naive" 模式快取
!curl -X POST "http://localhost:9621/documents/clear_cache" \
     -H "Content-Type: application/json" \
     -d '{ "modes": ["default", "naive"] }'

## query

### Query Text
`http://localhost:9621/query`  
用於接收使用者的查詢，並透過 RAG 檢索相關內容後生成回應。  
可以調整檢索模式，與各類參數(ex. rerank)。  

**body 參數**  
* `query`(required): 提問內容。格式：strings.
* 其餘參數參閱下方表格。
  
**回傳內容**  
* `response`(required): 生成的文字回覆，或是若快取中存有一樣的提問，則直接返回字串結果。


| 參數名稱                   | 型別      | 必填 | 說明                                                                                            |
| ---------------------- | ------- | -- | --------------------------------------------------------------------------------------------- |
| `query`                | string  | ✔  | 使用者的查詢內容                                                                                      |
| `mode`                 | string  | ✘  | 查詢模式（local、global、hybrid、naive、mix、bypass），預設 `"mix"`                                         |
| `only_need_context`    | boolean | ✘  | 若為 true，只回傳檢索到的上下文，不生成回應                                                                      |
| `only_need_prompt`     | boolean | ✘  | 若為 true，只回傳 prompt，不生成回應                                                                      |
| `response_type`        | string  | ✘  | 回應格式，例如 `"Multiple Paragraphs"`、`"Single Paragraph"`                                          |
| `top_k`                | integer | ✘  | 要檢索的前 K 筆項目數                                                                                  |
| `chunk_top_k`          | integer | ✘  | 初步檢索的 chunk 數量                                                                                |
| `max_entity_tokens`    | integer | ✘  | 實體分配的最大 token 數                                                                               |
| `max_relation_tokens`  | integer | ✘  | 關係分配的最大 token 數                                                                               |
| `max_total_tokens`     | integer | ✘  | 整個查詢 context 的 token 預算上限                                                                     |
| `conversation_history` | array   | ✘  | 用來保持對話連續性，格式如 `[{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]` |
| `history_turns`        | integer | ✘  | 要納入考慮的歷史對話輪數                                                                                  |
| `ids`                  | array   | ✘  | 過濾指定文件 ID                                                                                     |
| `user_prompt`          | string  | ✘  | 自訂提示詞，用來替代預設 prompt 模板                                                                        |
| `enable_rerank`        | boolean | ✘  | 是否啟用 rerank 功能，預設 `true`                                                                      |

In [18]:
!curl -X POST "http://lightrag:9621/query" \
     -H "Content-Type: application/json" \
     -d '{ "query": "解釋重力在流體力學中的作用",\
           "mode": "global",\
           "only_need_context": true,\
           "response_type": "Multiple Paragraphs",\
           "top_k": 40,\
           "chunk_top_k": 10,\
           "max_entity_tokens": 4000,\
           "max_relation_tokens": 4000,\
           "max_total_tokens": 10000,\
           "history_turns": 2,\
           "enable_rerank": true\
         }' | jq


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34863  100 34447  100   416   154k   1906 --:--:-- --:--:-- --:--:--  156k
{
  "response": "-----Entities(KG)-----\n\n```json\n[{\"id\": 1, \"entity\": \"壓力\", \"type\": \"category\", \"description\": \"Pressure is a physical quantity that represents the force exerted per unit area.\", \"created_at\": \"2025-08-05 08:21:32\", \"file_path\": \"fluid_mechanics.txt\"}, {\"id\": 2, \"entity\": \"流體力學\", \"type\": \"category\", \"description\": \"Fluid mechanics is a branch of physics that studies the behavior of liquids and gases, including their forces, pressure, buoyancy, and viscosity.\", \"created_at\": \"2025-08-05 08:21:32\", \"file_path\": \"fluid_mechanics.txt\"}, {\"id\": 3, \"entity\": \"液體\", \"type\": \"category\", \"description\": \"Liquids are a state of matter characterized by their ability to flow and take the shap

In [19]:
!curl -X POST "http://lightrag:9621/query" \
     -H "Content-Type: application/json" \
     -d '{ "query": "解釋重力在流體力學中的作用",\
           "mode": "global",\
           "only_need_context": false,\
           "response_type": "Multiple Paragraphs",\
           "top_k": 40,\
           "chunk_top_k": 10,\
           "max_entity_tokens": 4000,\
           "max_relation_tokens": 4000,\
           "max_total_tokens": 10000,\
           "history_turns": 2,\
           "enable_rerank": true\
         }' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1660  100  1243  100   417     84     28  0:00:14  0:00:14 --:--:--   351
{
  "response": "流體力學研究液體和氣體的運動與作用力，而重力在其中扮演著重要的角色。重力是物體之間由質量所產生的相互吸引力，它影響著行星的運行，並是導致自由落體運動的原因。\n\n在流體力學中，重力會產生壓力，並影響浮力和黏滯力等現象。例如，液體和氣體在重力作用下會產生壓力，壓力的大小取決於流體的密度和深度。浮力則是流體對浸入其中的物體產生的向上作用力，它的大小取決於流體的密度和物體的體積，以及重力加速度。黏滯力則是流體內摩擦力，它會阻礙流體的運動。\n\n此外，重力還會影響流體的穩定性。例如，在層流中，流體各層之間會受到重力的作用，使得密度較大的流體位於下方，密度較小的流體位於上方。而在湍流中，重力會產生渦流和混合作用，使得流體各層之間發生劇烈的混合。\n\n總之，重力是流體力學中一個重要的作用力，它影響著流體的運動、穩定性和各種物理現象。\n\nReferences:\n\n[KG] physics_gravity.txt\n[KG] fluid_mechanics.txt\n[DC] fluid_mechanics.txt\n[DC] little_prince_7.txt\n[DC] little_prince_27.txt"
}


In [30]:
# 未 rerank, local
!curl -X POST "http://lightrag:9621/query" \
     -H "Content-Type: application/json" \
     -d '{ "query": "羊存在的意義是什麼?",\
           "mode": "local",\
           "only_need_context": true,\
           "response_type": "Multiple Paragraphs",\
           "top_k": 40,\
           "chunk_top_k": 10,\
           "max_entity_tokens": 4000,\
           "max_relation_tokens": 4000,\
           "max_total_tokens": 10000,\
           "history_turns": 2,\
           "enable_rerank": false\
         }' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 32066  100 31661  100   405   3351     42  0:00:09  0:00:09 --:--:--  7470
{
  "response": "-----Entities(KG)-----\n\n```json\n[{\"id\": 1, \"entity\": \"綿羊\", \"type\": \"category\", \"description\": \"The sheep, or 绵羊 (Mián yáng), play a significant, recurring role in the story, particularly in relation to the Little Prince. They are more than just animals; they function as a central motif representing both potential danger to the Little Prince’s rose, and simultaneously, innocence and vulnerability as a possession he cares for deeply. The Little Prince repeatedly requests a muzzle for the sheep, demonstrating his awareness of its potential to harm his flower and his desire to protect it. Furthermore, the sheep are depicted as being inside a 箱子 (box), suggesting they represent something the narrator is attempting to understa

In [43]:
# rerank, local
!curl -X POST "http://lightrag:9621/query" \
     -H "Content-Type: application/json" \
     -d '{ "query": "羊存在的意義是什麼",\
           "mode": "local",\
           "only_need_context": true,\
           "response_type": "Multiple Paragraphs",\
           "top_k": 40,\
           "chunk_top_k": 10,\
           "max_entity_tokens": 4000,\
           "max_relation_tokens": 4000,\
           "max_total_tokens": 10000,\
           "history_turns": 2,\
           "enable_rerank": true\
         }' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35397  100 34994  100   403   3526     40  0:00:10  0:00:09  0:00:01  9417
{
  "response": "-----Entities(KG)-----\n\n```json\n[{\"id\": 1, \"entity\": \"綿羊\", \"type\": \"category\", \"description\": \"The sheep, or 绵羊 (Mián yáng), play a significant, recurring role in the story, particularly in relation to the Little Prince. They are more than just animals; they function as a central motif representing both potential danger to the Little Prince’s rose, and simultaneously, innocence and vulnerability as a possession he cares for deeply. The Little Prince repeatedly requests a muzzle for the sheep, demonstrating his awareness of its potential to harm his flower and his desire to protect it. Furthermore, the sheep are depicted as being inside a 箱子 (box), suggesting they represent something the narrator is attempting to understa

local 未 rerank  
{"response":"## 綿羊在故事中的意義\n\n在《小王子》的故事中，綿羊具有多重意義。綿羊不僅僅是動物，更代表了小王子對珍視之物的保護意識和潛在的危險。小王子反覆要求畫家為他畫一隻綿羊，並擔心綿羊會吃掉他的玫瑰花，這反映了他對心愛之物的謹慎和保護欲。\n\n綿羊的存在也暗示著一種潛在的危險。小王子害怕綿羊會破壞他的玫瑰花，因此他要求給綿羊戴上口 muzzle，以防止它造成傷害。這種擔憂體現了他對脆弱事物的保護意識，以及對潛在威脅的警惕。\n\n此外，綿羊在故事中也象征著想象力和創造力。小王子最初不滿意畫家畫的綿羊，因為他想要一隻能夠滿足他需求的綿羊。最終，畫家畫了一隻在箱子裡面的綿羊，這滿足了小王子的要求，也體現了想象力的力量。\n\n總而言之，綿羊在故事中不僅僅是一種動物，更代表了保護、危險和想象力等多种象征意义。 它與小王子的玫瑰花息息相關，共同構成了故事中重要的情感紐帶。\n\n**References:**\n\n* [KG] little_prince_2.txt\n* [KG] little_prince_3.txt\n* [KG] little_prince_7.txt\n* [KG] little_prince_25.txt\n* [KG] little_prince_27.txt"}

local rerank  
{"response":"## 綿羊在故事中的意義\n\n在《小王子》的故事中，綿羊具有多重意義。綿羊不僅僅是動物，更代表了小王子對珍視之物的保護意識和潛在的危險。小王子反覆要求畫家為他畫一隻綿羊，並擔心綿羊會吃掉他的玫瑰花，這反映了他對心愛之物的謹慎和保護欲。\n\n綿羊的存在也暗示著一種潛在的危險。小王子害怕綿羊會破壞他的玫瑰花，因此他要求給綿羊戴上口 muzzle，以防止它造成傷害。這種擔憂體現了他對脆弱事物的保護意識，以及對潛在威脅的警惕。\n\n此外，綿羊在故事中也象征著想象力和創造力。小王子最初不滿意畫家畫的綿羊，因為他想要一隻能夠滿足他需求的綿羊。最終，畫家畫了一隻在箱子裡面的綿羊，這滿足了小王子的要求，也體現了想象力的力量。\n\n總而言之，綿羊在故事中不僅僅是一種動物，更代表了保護、危險和想象力等多种象征意义。 它與小王子的玫瑰花息息相關，共同構成了故事中重要的情感紐帶。\n\n**References:**\n\n* [KG] little_prince_2.txt\n* [KG] little_prince_3.txt\n* [KG] little_prince_7.txt\n* [KG] little_prince_25.txt\n* [KG] little_prince_27.txt"}

In [33]:
# 未 rerank, global
!curl -X POST "http://lightrag:9621/query" \
     -H "Content-Type: application/json" \
     -d '{ "query": "羊存在的意義是什麼?",\
           "mode": "global",\
           "only_need_context": true,\
           "response_type": "Multiple Paragraphs",\
           "top_k": 40,\
           "chunk_top_k": 10,\
           "max_entity_tokens": 4000,\
           "max_relation_tokens": 4000,\
           "max_total_tokens": 10000,\
           "history_turns": 2,\
           "enable_rerank": false\
         }'

{"response":"-----Entities(KG)-----\n\n```json\n[{\"id\": 1, \"entity\": \"綁繩\", \"type\": \"category\", \"description\": \"The rope is intended to tie up the sheep, representing a practical solution proposed by the narrator.\", \"created_at\": \"2025-08-05 06:50:05\", \"file_path\": \"little_prince_3.txt\"}, {\"id\": 2, \"entity\": \"綿羊\", \"type\": \"category\", \"description\": \"The sheep, or 绵羊 (Mián yáng), play a significant, recurring role in the story, particularly in relation to the Little Prince. They are more than just animals; they function as a central motif representing both potential danger to the Little Prince’s rose, and simultaneously, innocence and vulnerability as a possession he cares for deeply. The Little Prince repeatedly requests a muzzle for the sheep, demonstrating his awareness of its potential to harm his flower and his desire to protect it. Furthermore, the sheep are depicted as being inside a 箱子 (box), suggesting they represent something the narrator is a

In [42]:
# rerank, global
!curl -X POST "http://lightrag:9621/query" \
     -H "Content-Type: application/json" \
     -d '{ "query": "羊存在的意義是什?",\
           "mode": "global",\
           "only_need_context": true,\
           "response_type": "Multiple Paragraphs",\
           "top_k": 40,\
           "chunk_top_k": 10,\
           "max_entity_tokens": 4000,\
           "max_relation_tokens": 4000,\
           "max_total_tokens": 10000,\
           "history_turns": 1,\
           "enable_rerank": true\
         }'  | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34366  100 33964  100   402  15862    187  0:00:02  0:00:02 --:--:-- 16051
{
  "response": "-----Entities(KG)-----\n\n```json\n[{\"id\": 1, \"entity\": \"綁繩\", \"type\": \"category\", \"description\": \"The rope is intended to tie up the sheep, representing a practical solution proposed by the narrator.\", \"created_at\": \"2025-08-05 06:50:05\", \"file_path\": \"little_prince_3.txt\"}, {\"id\": 2, \"entity\": \"綿羊\", \"type\": \"category\", \"description\": \"The sheep, or 绵羊 (Mián yáng), play a significant, recurring role in the story, particularly in relation to the Little Prince. They are more than just animals; they function as a central motif representing both potential danger to the Little Prince’s rose, and simultaneously, innocence and vulnerability as a possession he cares for deeply. The Little Prince repeatedly req

未 rerank, global  
"## The Significance of Sheep in *The Little Prince*\n\nWithin the context of *The Little Prince*, sheep hold a multifaceted significance, primarily revolving around themes of care, responsibility, and the potential for both beauty and destruction. The Little Prince repeatedly requests a muzzle for the sheep, demonstrating his deep concern for protecting his rose. This isn’t simply about controlling the sheep’s movement, but about safeguarding something precious from potential harm. The sheep, therefore, symbolizes a potential danger to the rose, while simultaneously representing innocence and vulnerability as a possession the Little Prince deeply cares for. \n\nThe interaction surrounding the sheep also reveals differing perspectives on responsibility. The narrator suggests tying up the sheep to prevent it from running away, a practical solution that the Little Prince finds absurd. This highlights a contrast between pragmatic control and the Little Prince’s desire for freedom and natural existence, even if it entails risk. The Little Prince's questioning of where the sheep would even *go* if not contained underscores his small world and the inherent limitations within it. The sheep, existing within a box (箱子), further suggests they represent something the narrator is attempting to understand or portray. \n\nUltimately, the sheep aren't merely animals; they’re a central motif representing the need for careful attention and protection of what is cherished. They serve as a catalyst for discussions about responsibility, care, and the delicate balance between freedom and security within the narrative.\n\n**References:**\n\n* [KG] little_prince_3.txt\n* [KG] little_prince_27.txt\n* [KG] little_prince_2.txt\n* [KG] little_prince_4.txt\n* [KG] little_prince_7.txt\n"

rerank, global  
{"response":"## 羊的意義\n\n根據所提供的知識庫，羊在故事中扮演了多重角色，並與小王子、敘述者以及故事的整體主題息息相關。\n\n首先，羊是小王子所關心和請求保護的對象之一。小王子害怕羊會吃掉他的玫瑰花，因此一直要求敘述者為羊畫上口套，以保護玫瑰花。這突顯了小王子對所愛之物的責任感和保護欲。\n\n其次，羊也代表著一種需要被引導和控制的對象。敘述者建議將羊綁起來，以防止牠走失，但小王子認為這毫無意義，因為羊所處的空間非常有限。這暗示了小王子對自由和個性的重視，以及對過度控制的反對。\n\n此外，羊還與故事中關於責任、關懷和馴養的主題密切相關。小王子對羊的關心和保護，以及他對馴養的理解，都反映了他對生命和情感的深刻體悟。\n\n總而言之，羊在故事中不僅僅是一種動物，更是一種象徵，代表著關懷、責任、自由以及馴養等重要的主題。\n\n**References:**\n\n*   [KG] little_prince_3.txt\n*   [KG] little_prince_2.txt\n*   [KG] little_prince_25.txt\n*   [KG] little_prince_7.txt\n*   [KG] little_prince_21.txt"}

### Query Text Stream
`http://localhost:9621/query/stream`  
從 RAG 系統串流式獲取回應。

**body 參數**  
* `query`(required): 提問內容。格式：strings.
* 其餘參數參閱下方表格。
  
**回傳內容**  
* `response`(required): 串流輸出的回應結果。


| 參數名稱                   | 型別      | 必填 | 預設值        | 說明                                                            |
| ---------------------- | ------- | -- | ---------- | ------------------------------------------------------------- |
| `query`                | string  | ✔  | 無          | 使用者的查詢文字                                                      |
| `mode`                 | string  | ✘  | `"mix"`    | 查詢模式，可選：`local`, `global`, `hybrid`, `naive`, `mix`, `bypass` |
| `only_need_context`    | boolean | ✘  | false      | 若為 true，只回傳檢索到的 context，不生成回答                                 |
| `only_need_prompt`     | boolean | ✘  | false      | 若為 true，只回傳生成的 prompt，不生成回答                                   |
| `response_type`        | string  | ✘  | `"string"` | 回應格式，如 `"paragraph"`, `"bullet_points"`                       |
| `top_k`                | int     | ✘  | 3          | 檢索前 K 個文件或片段                                                  |
| `chunk_top_k`          | int     | ✘  | 2          | 片段級別檢索數量                                                      |
| `max_entity_tokens`    | int     | ✘  | 128        | 分配給實體控制的最大 tokens                                             |
| `max_relation_tokens`  | int     | ✘  | 128        | 分配給關係控制的最大 tokens                                             |
| `max_total_tokens`     | int     | ✘  | 512        | 查詢上下文 + 系統提示 + 實體關係的最大 token 數                                |
| `conversation_history` | array   | ✘  | \[]        | 對話歷史，格式：`[{"role":"user","content":"..."}]`                   |
| `history_turns`        | int     | ✘  | 0          | 考慮的對話輪數                                                       |
| `ids`                  | array   | ✘  | \[]        | 文件 ID 過濾                                                      |
| `user_prompt`          | string  | ✘  | 無          | 自定義 prompt                                                    |
| `enable_rerank`        | boolean | ✘  | true       | 是否啟用 rerank                                                   |

In [23]:
!curl -X POST "http://lightrag:9621/query/stream" \
     -H "Content-Type: application/json" \
     -d '{ "query": "解釋流體力學中的浮力原理",\
           "mode": "local",\
           "only_need_context": true,\
           "response_type": "Multiple Paragraphs",\
           "top_k": 40,\
           "chunk_top_k": 10,\
           "max_entity_tokens": 4000,\
           "max_relation_tokens": 4000,\
           "max_total_tokens": 10000,\
           "history_turns": 2,\
           "enable_rerank": true\
         }' | jq


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37579    0 37167  100   412  14374    159  0:00:02  0:00:02 --:--:-- 14531
{
  "response": "-----Entities(KG)-----\n\n```json\n[{\"id\": 1, \"entity\": \"浮力\", \"type\": \"category\", \"description\": \"Buoyancy is the upward force exerted by a fluid that opposes the weight of an immersed object.\", \"created_at\": \"2025-08-05 08:21:32\", \"file_path\": \"fluid_mechanics.txt\"}, {\"id\": 2, \"entity\": \"质量\", \"type\": \"category\", \"description\": \"质量是物体所具有的固有属性，是产生重力的根本原因。\", \"created_at\": \"2025-08-05 08:21:25\", \"file_path\": \"physics_gravity.txt\"}, {\"id\": 3, \"entity\": \"重力\", \"type\": \"category\", \"description\": \"重力是物體之間由質量所產生的相互吸引力，影響行星運行與自由落體運動。\", \"created_at\": \"2025-08-05 08:21:25\", \"file_path\": \"physics_gravity.txt\"}, {\"id\": 4, \"entity\": \"物体\", \"type\": \"category\", \"description\": \

In [24]:
!curl -X POST "http://lightrag:9621/query/stream" \
     -H "Content-Type: application/json" \
     -d '{ "query": "解釋流體力學中的浮力原理",\
           "mode": "global",\
           "only_need_context": true,\
           "response_type": "Multiple Paragraphs",\
           "top_k": 40,\
           "chunk_top_k": 10,\
           "max_entity_tokens": 4000,\
           "max_relation_tokens": 4000,\
           "max_total_tokens": 10000,\
           "history_turns": 2,\
           "enable_rerank": true\
         }' | jq


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37774    0 37361  100   413  16650    184  0:00:02  0:00:02 --:--:-- 16840
{
  "response": "-----Entities(KG)-----\n\n```json\n[{\"id\": 1, \"entity\": \"流體力學\", \"type\": \"category\", \"description\": \"Fluid mechanics is a branch of physics that studies the behavior of liquids and gases, including their forces, pressure, buoyancy, and viscosity.\", \"created_at\": \"2025-08-05 08:21:32\", \"file_path\": \"fluid_mechanics.txt\"}, {\"id\": 2, \"entity\": \"液體\", \"type\": \"category\", \"description\": \"Liquids are a state of matter characterized by their ability to flow and take the shape of their container.\", \"created_at\": \"2025-08-05 08:21:32\", \"file_path\": \"fluid_mechanics.txt\"}, {\"id\": 3, \"entity\": \"氣體\", \"type\": \"category\", \"description\": \"Gases are a state of matter characterized by their ability 

In [27]:
!curl -X POST "http://lightrag:9621/query/stream" \
     -H "Content-Type: application/json" \
     -d '{ "query": "解釋流體力學中的浮力原理",\
           "mode": "global",\
           "only_need_context": false,\
           "response_type": "Multiple Paragraphs",\
           "top_k": 40,\
           "chunk_top_k": 10,\
           "max_entity_tokens": 4000,\
           "max_relation_tokens": 4000,\
           "max_total_tokens": 10000,\
           "history_turns": 2,\
           "enable_rerank": true\
         }' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4321    0  3907  100   414    374     39  0:00:10  0:00:10 --:--:--   901{
  "response": "##"
}
{
  "response": " 流"
}
{
  "response": "體"
}
{
  "response": "力"
}
{
  "response": "學"
}
{
  "response": "中的"
}
{
  "response": "浮"
}
{
  "response": "力"
}
{
  "response": "原理"
}
{
  "response": "\n\n"
}
{
  "response": "流"
}
{
  "response": "體"
}
{
  "response": "力"
}
{
  "response": "學"
}
{
  "response": "是"
}
{
  "response": "研究"
}
{
  "response": "液"
}
{
  "response": "體"
}
{
  "response": "與"
}
{
  "response": "氣"
}
{
  "response": "體"
}
{
  "response": "運動"
}
{
  "response": "及"
}
{
  "response": "作用"
}
{
  "response": "力的"
}
{
  "response": "物理"
}
{
  "response": "學"
}
{
  "response": "分支"
}
{
  "response": "。"
}
{
  "response": "其中"
}
{
  "response": "，"
}
{
  "response": "浮"
}
{
  "response": "力"
}
{
  "response": "是"
}
{


In [28]:
!curl -X POST "http://lightrag:9621/query/stream" \
     -H "Content-Type: application/json" \
     -d '{ "query": "解釋流體力學中的浮力原理",\
           "mode": "global",\
           "only_need_context": true,\
           "response_type": "Multiple Paragraphs",\
           "top_k": 40,\
           "chunk_top_k": 10,\
           "max_entity_tokens": 4000,\
           "max_relation_tokens": 4000,\
           "max_total_tokens": 10000,\
           "history_turns": 2,\
           "enable_rerank": false\
         }' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37775    0 37361  100   414   132k   1506 --:--:-- --:--:-- --:--:--  134k
{
  "response": "-----Entities(KG)-----\n\n```json\n[{\"id\": 1, \"entity\": \"流體力學\", \"type\": \"category\", \"description\": \"Fluid mechanics is a branch of physics that studies the behavior of liquids and gases, including their forces, pressure, buoyancy, and viscosity.\", \"created_at\": \"2025-08-05 08:21:32\", \"file_path\": \"fluid_mechanics.txt\"}, {\"id\": 2, \"entity\": \"液體\", \"type\": \"category\", \"description\": \"Liquids are a state of matter characterized by their ability to flow and take the shape of their container.\", \"created_at\": \"2025-08-05 08:21:32\", \"file_path\": \"fluid_mechanics.txt\"}, {\"id\": 3, \"entity\": \"氣體\", \"type\": \"category\", \"description\": \"Gases are a state of matter characterized by their ability 

In [29]:
!curl -X POST "http://lightrag:9621/query/stream" \
     -H "Content-Type: application/json" \
     -d '{ "query": "解釋流體力學中的浮力原理",\
           "mode": "global",\
           "only_need_context": false,\
           "response_type": "Multiple Paragraphs",\
           "top_k": 40,\
           "chunk_top_k": 10,\
           "max_entity_tokens": 4000,\
           "max_relation_tokens": 4000,\
           "max_total_tokens": 10000,\
           "history_turns": 2,\
           "enable_rerank": false\
         }' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3868    0  3453  100   415    357     43  0:00:09  0:00:09 --:--:--   795{
  "response": "##"
}
{
  "response": " 流"
}
{
  "response": "體"
}
{
  "response": "力"
}
{
  "response": "學"
}
{
  "response": "中的"
}
{
  "response": "浮"
}
{
  "response": "力"
}
{
  "response": "原理"
}
{
  "response": "\n\n"
}
{
  "response": "流"
}
{
  "response": "體"
}
{
  "response": "力"
}
{
  "response": "學"
}
{
  "response": "是"
}
{
  "response": "研究"
}
{
  "response": "液"
}
{
  "response": "體"
}
{
  "response": "和"
}
{
  "response": "氣"
}
{
  "response": "體"
}
{
  "response": "運動"
}
{
  "response": "和"
}
{
  "response": "作用"
}
{
  "response": "力的"
}
{
  "response": "物理"
}
{
  "response": "學"
}
{
  "response": "分支"
}
{
  "response": "。"
}
{
  "response": "在"
}
{
  "response": "流"
}
{
  "response": "體"
}
{
  "response": "力"
}
{
  "response": "學"
}
{
 

## graph


### Get Graph Labels
`http://localhost:9621/graph/label/list`  
用於取得系統中所有圖譜標籤，若當前系統中沒有標籤則會回傳 `null` 或空陣列。  

**回傳內容**  
* 標籤: string 字串。  

In [1]:
!curl http://lightrag:9621/graph/label/list

[" narrator ","1444次落日","24小时","Agitation","Chinese","Emotions","English","Flowers","I","Japanese","Narrator","Person","She","The Baobabs","The Businessman","The Conceited Man","The Departure","The Desert","The Drunkard","The Fox","The Geographer","The Journey","The King","The Lamplighter","The Laughter","The Memory","The Narrator","The Night","The Other","The Other Person","The Other personified stars","The Other personified stars with whom narrator, the other entity, has a special bond","The Pilot","The Planet of the Little Prince","The Rose","The Roses","The Snake","The Star","The Starry Night","The Stars","The Switch","The Well","The Year","The flower","hello","narrator","こんにちは","中國","中國和西伯利亞","五萬萬","井","他","你好","光","六歲","其他玫瑰花","加法","動物","北極","北美洲","南極","南美洲","印度","发动机","另一個星球","商人","問題","四十三次","回聲","國王","國際天文學會","土壤","土耳其天文學家","地球","地理學家","壓力","夜晚的涼風","大人","大人們","太阳","太陽","她","好虛榮的人","宇宙","實業家","小王子","小王子&花","小王子行星","小盒子","小鈴子","小麥","屏風","島","工具","巴欧巴","巴歐巴","巴歐巴幼苗","憂鬱","我","撒哈拉

In [2]:
!curl -X GET "http://lightrag:9621/graph/label/list" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2558  100  2558    0     0   716k      0 --:--:-- --:--:-- --:--:--  832k
[
  " narrator ",
  "1444次落日",
  "24小时",
  "Agitation",
  "Chinese",
  "Emotions",
  "English",
  "Flowers",
  "I",
  "Japanese",
  "Narrator",
  "Person",
  "She",
  "The Baobabs",
  "The Businessman",
  "The Conceited Man",
  "The Departure",
  "The Desert",
  "The Drunkard",
  "The Fox",
  "The Geographer",
  "The Journey",
  "The King",
  "The Lamplighter",
  "The Laughter",
  "The Memory",
  "The Narrator",
  "The Night",
  "The Other",
  "The Other Person",
  "The Other personified stars",
  "The Other personified stars with whom narrator, the other entity, has a special bond",
  "The Pilot",
  "The Planet of the Little Prince",
  "The Rose",
  "The Roses",
  "The Snake",
  "The Star",
  "The Starry Night",
  "The Stars",
  "The Switch",
  "The We

### Get Knowledge Graph
`http://localhost:9621/graphs`  
用於檢索知識圖譜的子圖，會依據指定的 label 作為起點，回傳一個相連的節點子圖，可使用 `max_nodes` 與 `max_depth` 限制輸出節點數量或層級深度。

**傳入參數**  
| 名稱                      | 型別            | 必填  | 預設值  | 說明                        |
| ----------------------- | ------------- | --- | ---- | ------------------------- |
| label                   | string        | ✔ | 無    | 作為子圖起點的標籤名稱               |
| max\_depth              | integer       | ✘   | 3    | 搜索子圖的最大深度 (hop 數)         |
| max\_nodes              | integer       | ✘   | 1000 | 回傳的最大節點數量                 |
| api\_key\_header\_value | string / null | ✘   | 無    | API Key Header，用於驗證 (若需要) |

**回傳內容**  
* 標籤: string 字串。


In [3]:
# 點燈人 UTF-8 URL 編碼 = %E9%BB%9E%E7%87%88%E4%BA%BA
!curl -X GET "http://lightrag:9621/graphs?label=%E9%BB%9E%E7%87%88%E4%BA%BA&max_depth=1&max_nodes=10" \
     -H "Content-Type: application/json" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7233  100  7233    0     0  1835k      0 --:--:-- --:--:-- --:--:-- 2354k
{
  "nodes": [
    {
      "id": "小王子",
      "labels": [
        "小王子"
      ],
      "properties": {
        "entity_id": "小王子",
        "entity_type": "person",
        "description": "The Little Prince (小王子) is the central protagonist of the story, originating from a very small planet and preparing to eventually return to it after traveling between worlds. He is known for his inquisitive nature and unique perspectives, often differing from those of adults, and consistently seeks meaning in life through reflection on relationships and experiences. He feels a deep care and responsibility for his rose and deeply values simple pleasures and the beauty of hidden things. Throughout his journeys, and while sometimes experiencing melancholy, fear, or thirst

In [4]:
# 自動轉換編碼
import urllib.parse
label="星球"
encoded_label=urllib.parse.quote(label)

!curl -X GET "http://lightrag:9621/graphs?label=$encoded_label&max_depth=3&max_nodes=10" \
     -H "Content-Type: application/json" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   344  100   344    0     0  97560      0 --:--:-- --:--:-- --:--:--  111k
{
  "nodes": [
    {
      "id": "星球",
      "labels": [
        "星球"
      ],
      "properties": {
        "entity_id": "星球",
        "entity_type": "geo",
        "description": "Planets are referenced as the origins of both the narrator and the Little Prince.",
        "source_id": "chunk-88be53ee8f865cec9a3b7ca299abd998",
        "file_path": "little_prince_3.txt",
        "created_at": 1754376605
      }
    }
  ],
  "edges": [],
  "is_truncated": false
}


In [5]:
!echo $label

星球


In [6]:
!echo $encoded_label

%E6%98%9F%E7%90%83


### Check Entity Exists
`http://localhost:9621/graph/entity/exists`  
在進行查詢或建立關聯前，用於檢查知識圖譜中是否存在特定名稱的實體。  

**傳入參數**  
* `name (required)`: Entity name to check.
* `api_key_header_value`: 授權 API key

**回傳內容**  
* `true`: 該實體存在知識圖譜。
* `false`: 該實體不存在知識圖譜。


In [7]:
import urllib.parse
label="星球"
encoded_label=urllib.parse.quote(label)

!curl -X GET "http://lightrag:9621/graph/entity/exists?name=$encoded_label" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    15  100    15    0     0   5324      0 --:--:-- --:--:-- --:--:--  7500
{
  "exists": true
}


In [9]:
import urllib.parse
label="牛頓"
encoded_label=urllib.parse.quote(label)

!curl -X GET "http://lightrag:9621/graph/entity/exists?name=$encoded_label" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    16  100    16    0     0   5639      0 --:--:-- --:--:-- --:--:--  8000
{
  "exists": false
}


### Update Entity
`http://localhost:9621/graph/entity/edit`  
用於更新知識圖譜中指定實體(entity)的屬性，可修改實體的相關資料(ex.描述、分類), 可選擇是否允許重新命名實體名稱。  

**傳入參數**  
* `entity_name (required)`: Entity name to check.
* `updated_data (required)`: 授權 API key。
  * `property name`: 可額外添加任何額外的鍵值。
* `allow_rename`: 是否允許修改實體名稱。


In [15]:
# 自動轉換編碼
import urllib.parse
label="24小时"
encoded_label=urllib.parse.quote(label)

!curl -X GET "http://lightrag:9621/graphs?label=$encoded_label&max_depth=3&max_nodes=10" \
     -H "Content-Type: application/json" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   358  100   358    0     0   101k      0 --:--:-- --:--:-- --:--:--  116k
{
  "nodes": [
    {
      "id": "24小时",
      "labels": [
        "24小时"
      ],
      "properties": {
        "entity_id": "24小时",
        "entity_type": "category",
        "description": "A time duration of 24 hours, used to define the frequency of sunsets on the planet.",
        "source_id": "chunk-1e59e8759cd26e10025c9213dc1abed1",
        "file_path": "little_prince_14.txt",
        "created_at": 1754375985
      }
    }
  ],
  "edges": [],
  "is_truncated": false
}


In [17]:
!curl -X POST "http://lightrag:9621/graph/entity/edit" \
     -H "Content-Type: application/json" \
     -d '{ "entity_name": "24小时", \
           "updated_data": {\
             "description": "時間週期。",\
             "unit": "hours",\
             "category": "Time",\
             "related_concept": ["地球自轉", "一天", "時間"]\
           },\
           "allow_rename": false\
         }'


{"status":"success","message":"Entity updated successfully","data":{"entity_name":"24小时","source_id":"chunk-1e59e8759cd26e10025c9213dc1abed1","graph_data":{"entity_id":"24小时","entity_type":"category","description":"時間週期。","source_id":"chunk-1e59e8759cd26e10025c9213dc1abed1","file_path":"little_prince_14.txt","created_at":1754375985,"unit":"hours","category":"Time","related_concept":["地球自轉","一天","時間"]},"vector_data":{"__id__":"ent-c39e670828a272db4839f83c8810fed8","__created_at__":1754453665,"content":"24小时\n時間週期。","entity_name":"24小时","source_id":"chunk-1e59e8759cd26e10025c9213dc1abed1","id":"ent-c39e670828a272db4839f83c8810fed8","created_at":1754453665}}}

In [25]:
!curl -X POST "http://lightrag:9621/graph/entity/edit" \
     -H "Content-Type: application/json" \
     -d '{ "entity_name": "24小时", \
           "updated_data": { \
             "name": "24小時" \
           }, \
           "allow_rename": true \
         }'


{"status":"success","message":"Entity updated successfully","data":{"entity_name":"24小时","source_id":"chunk-1e59e8759cd26e10025c9213dc1abed1","graph_data":{"entity_id":"24小时","entity_type":"category","description":"時間週期。","source_id":"chunk-1e59e8759cd26e10025c9213dc1abed1","file_path":"little_prince_14.txt","created_at":1754375985,"unit":"hours","category":"Time","related_concept":["地球自轉","一天","時間"],"new_name":"24小時","name":"24小時"},"vector_data":{"__id__":"ent-c39e670828a272db4839f83c8810fed8","__created_at__":1754454577,"content":"24小时\n時間週期。","entity_name":"24小时","source_id":"chunk-1e59e8759cd26e10025c9213dc1abed1","id":"ent-c39e670828a272db4839f83c8810fed8","created_at":1754454577}}}

執行後
# 插入圖片(還未插入)

24_hour_node.PNG


In [18]:
# 自動轉換編碼
import urllib.parse
label="24小时"
encoded_label=urllib.parse.quote(label)

!curl -X GET "http://lightrag:9621/graphs?label=$encoded_label&max_depth=3&max_nodes=10" \
     -H "Content-Type: application/json" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   376  100   376    0     0   107k      0 --:--:-- --:--:-- --:--:--  122k
{
  "nodes": [
    {
      "id": "24小时",
      "labels": [
        "24小时"
      ],
      "properties": {
        "entity_id": "24小时",
        "entity_type": "category",
        "description": "時間週期。",
        "source_id": "chunk-1e59e8759cd26e10025c9213dc1abed1",
        "file_path": "little_prince_14.txt",
        "created_at": 1754375985,
        "unit": "hours",
        "category": "Time",
        "related_concept": [
          "地球自轉",
          "一天",
          "時間"
        ]
      }
    }
  ],
  "edges": [],
  "is_truncated": false
}


In [26]:
# 自動轉換編碼
import urllib.parse
label="24小时"
encoded_label=urllib.parse.quote(label)

!curl -X GET "http://lightrag:9621/graphs?label=$encoded_label&max_depth=3&max_nodes=10" \
     -H "Content-Type: application/json" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   416  100   416    0     0   124k      0 --:--:-- --:--:-- --:--:--  135k
{
  "nodes": [
    {
      "id": "24小时",
      "labels": [
        "24小时"
      ],
      "properties": {
        "entity_id": "24小时",
        "entity_type": "category",
        "description": "時間週期。",
        "source_id": "chunk-1e59e8759cd26e10025c9213dc1abed1",
        "file_path": "little_prince_14.txt",
        "created_at": 1754375985,
        "unit": "hours",
        "category": "Time",
        "related_concept": [
          "地球自轉",
          "一天",
          "時間"
        ],
        "new_name": "24小時",
        "name": "24小時"
      }
    }
  ],
  "edges": [],
  "is_truncated": false
}


### Update Relation
`http://localhost:9621/graph/relation/edit`  
用於更新知識圖譜中兩個 entity 間的關係屬性、描述。


**傳入參數**  
* `source_id (required)`: 關係來源實體的 ID。
* `target_id (required)`: 關係目標實體的 ID。
* `updated_data (required)`: 要更新的關係屬性。

<!-- **回傳內容**  
* `true`: 該實體存在知識圖譜。
* `false`: 該實體不存在知識圖譜。 -->

In [27]:
!curl -X POST "http://lightrag:9621/graph/relation/edit" \
     -H "Content-Type: application/json" \
     -d '{ "source_id": "太阳", \
           "target_id": "24小时", \
           "updated_data": { \
             "relation_type": "cause", \
             "description": "地球自轉導致24小時形成。" \
           } \
         }'

{"detail":"Relation from '太阳' to '24小时' does not exist"}

In [29]:
!curl -X POST "http://lightrag:9621/graph/relation/edit" \
     -H "Content-Type: application/json" \
     -d '{ "source_id": "玫瑰花", \
           "target_id": "花園", \
           "updated_data": { \
             "relation_type": "live in", \
             "description": "玫瑰們居住在花園中。" \
           } \
         }'


{"status":"success","message":"Relation updated successfully","data":{"src_entity":"玫瑰花","tgt_entity":"花園","source_id":"chunk-0b01b705d8958985aabe44758c6accd4","graph_data":{"weight":7.0,"description":"玫瑰們居住在花園中。","keywords":"abundance,contrast,similarity","source_id":"chunk-0b01b705d8958985aabe44758c6accd4","file_path":"little_prince_20.txt","created_at":1754376163,"relation_type":"live in"},"vector_data":{"__id__":"rel-1d7ce08eee0055b8a075eadaab0bd3d8","__created_at__":1754460566,"content":"玫瑰花\t花園\nabundance,contrast,similarity\n玫瑰們居住在花園中。","src_id":"玫瑰花","tgt_id":"花園","source_id":"chunk-0b01b705d8958985aabe44758c6accd4","id":"rel-1d7ce08eee0055b8a075eadaab0bd3d8","created_at":1754460566}}}

執行後
# 插入圖片(還未插入)
rose_garden.PNG


## ollama

### Chat
`http://localhost:9621/api/chat`  
此端點會處理使用者的對話請求，並透過 LightRAG 路由至 Ollama 模型。

In [35]:
!curl -X POST "http://lightrag:9621/api/chat" \
     -H "Content-Type: application/json" \
     -d '{ "model": "gemma3:27b-max-content", \
           "messages": [ \
             {"role": "user", "content": "什麼是重力？"}, \
             {"role": "assistant", "content": "重力是地球吸引物體的力。"}, \
             {"role": "user", "content": "解釋重力與浮力的差異"} \
           ] \
         }'


{"model": "lightrag:latest", "created_at": "2024-01-15T00:00:00Z", "message": {"role": "assistant", "content": "好的", "images": null}, "done": false}
{"model": "lightrag:latest", "created_at": "2024-01-15T00:00:00Z", "message": {"role": "assistant", "content": "，", "images": null}, "done": false}
{"model": "lightrag:latest", "created_at": "2024-01-15T00:00:00Z", "message": {"role": "assistant", "content": "我們", "images": null}, "done": false}
{"model": "lightrag:latest", "created_at": "2024-01-15T00:00:00Z", "message": {"role": "assistant", "content": "來", "images": null}, "done": false}
{"model": "lightrag:latest", "created_at": "2024-01-15T00:00:00Z", "message": {"role": "assistant", "content": "解釋", "images": null}, "done": false}
{"model": "lightrag:latest", "created_at": "2024-01-15T00:00:00Z", "message": {"role": "assistant", "content": "一下", "images": null}, "done": false}
{"model": "lightrag:latest", "created_at": "2024-01-15T00:00:00Z", "message": {"role": "assistant", "content

`GET`:
 * `/api/version`  
   http://localhost:9621/api/version
 * `/api/tags`  
   http://localhost:9621/api/tags
 * `/api/ps`  
   http://localhost:9621/api/ps
   
`POST`:
 * `/api/generate`  
   http://localhost:9621/api/generate
 * `/api/chat`  
   http://localhost:9621/api/chat




In [ ]:
!curl -X POST "http://lightrag:9621/documents/text" \
    -H "Content-Type: application/json" \
    -d '{"text": "細胞是所有生物體的基本單位，所有生物體的結構和功能都離不開細胞。根據細胞的結構，可以將其分為兩類：原核細胞和真核細胞。原核細胞如細菌和藍綠藻，沒有明確的細胞核，遺傳物質直接位於細胞質中。而真核細胞則具有明確的細胞核，並且擁有分工更細的細胞器，如線粒體、內質網等。每個細胞內都包含一個基因組，負責遺傳訊息的儲存與傳遞。細胞的生命活動依賴於複雜的化學反應，這些反應在細胞內的各種細胞器中進行。"}' # description= option description

In [ ]:
!curl -X POST "http://localhost:9621/documents/scan" \
     -H "Content-Type: application/json" \ # 告知傳送格式內容
     -d '{}'# 不需要額外內容

### POST /query  
使用不同搜尋模式查詢 RAG。

**模式**  
* native
* local
* global
* hybrid
* mix
* bypass

In [1]:
!curl -X POST "http://lightrag:9621/query" \
    -H "Content-Type: application/json" \
    -d '{"query": "林致遠最後的心願是什麼？他是否如願？", "mode": "hybrid"}'

{"response":"## 林致遠的最後心願\n\n林致遠在生命最後期，最大的願望是能看到愛與知識的傳承。他希望有人能接手知一書屋，延續他對書籍和文學的热爱。在與蘇曉的交往中，他逐漸將書店託付給蘇曉，並希望她能將愛與知識傳承下去 (DC: cat_story_2.txt)。\n\n從故事的結尾來看，林致遠的心願得以實現。蘇曉接手了知一書屋，並將其經營成當地最具人氣的文學書店 (DC: cat_story_2.txt)。她不僅守護了這間書店，更將愛與知識傳遞給更多人。因此，林致遠不僅如願以償，也見證了愛與知識的永恆傳承。\n\n此外，林致遠也希望書店能繼續充滿生氣。在他生命的最後時期，蘇曉的出現為他帶來了慰藉與希望，使他重新燃起了對生活的熱情。書店也因此重煥生機，這也算得上是林致遠心願的實現 (DC: cat_story_2.txt)。\n\n## References\n\n* [KG] cat_story_2.txt\n* [DC] cat_story_2.txt\n* [DC] cat_story_1.txt\n* [KG] cat_story_1.txt\n* [KG] cat_story_2.txt"}

In [2]:
!curl -X POST "http://lightrag:9621/query" \
    -H "Content-Type: application/json" \
    -d '{"query": "林致遠最後的心願是什麼？他是否如願？", "mode": "global"}'

{"response":"## 林致遠的最後心願\n\n林致遠的最後心願是希望有人能將愛與知識傳承下去。他在臨終前寫給蘇曉的信中表達了這一願望，並將知一書屋留給了她，希望她能繼續經營書店，傳承愛與知識（DC cat_story_2.txt）。\n\n從故事的發展來看，林致遠的願望得以實現。蘇曉接手了知一書屋，並將其打造成了當地最具人氣的文學書店，繼續傳承著書中的愛與知識（DC cat_story_2.txt）。她還養了一隻黑貓，取名阿墨，似乎延續了林致遠與阿墨之間的緣分。\n\n此外，林致遠也希望有人能放下過去，學會愛與被愛，而蘇曉的出現，似乎也完成了他的這份期盼。阿墨也一直守護著人們，直到他們放下過去，學會愛與被愛（DC cat_story_2.txt）。\n\n## References:\n\n*   [DC] cat_story_2.txt\n*   [DC] cat_story_1.txt\n*   [KG] cat_story_2.txt\n*   [KG] cat_story_1.txt\n*   [KG] cat_story_2.txt;cat_story_1.txt"}

### POST /query/stream  
從 RAG 系統串流式獲取回應。

用於接收使用者的查詢，並透過 RAG 檢索相關內容後生成回應。
可以調整檢索模式，與各類參數(ex. rerank)。

In [3]:
# Unicode 編碼
!curl -X POST "http://lightrag:9621/query/stream" \
    -H "Content-Type: application/json" \
    -d '{"query": "林致遠最後的心願是什麼？他是否如願？", "mode": "global"}'

{"response": "## \u6797\u81f4\u9060\u7684\u6700\u5f8c\u5fc3\u9858\n\n\u6797\u81f4\u9060\u7684\u6700\u5f8c\u5fc3\u9858\u662f\u5e0c\u671b\u6709\u4eba\u80fd\u5c07\u611b\u8207\u77e5\u8b58\u50b3\u627f\u4e0b\u53bb\u3002\u4ed6\u5728\u81e8\u7d42\u524d\u5beb\u7d66\u8607\u66c9\u7684\u4fe1\u4e2d\u8868\u9054\u4e86\u9019\u4e00\u9858\u671b\uff0c\u4e26\u5c07\u77e5\u4e00\u66f8\u5c4b\u7559\u7d66\u4e86\u5979\uff0c\u5e0c\u671b\u5979\u80fd\u7e7c\u7e8c\u7d93\u71df\u66f8\u5e97\uff0c\u50b3\u627f\u611b\u8207\u77e5\u8b58\uff08DC cat_story_2.txt\uff09\u3002\n\n\u5f9e\u6545\u4e8b\u7684\u767c\u5c55\u4f86\u770b\uff0c\u6797\u81f4\u9060\u7684\u9858\u671b\u5f97\u4ee5\u5be6\u73fe\u3002\u8607\u66c9\u63a5\u624b\u4e86\u77e5\u4e00\u66f8\u5c4b\uff0c\u4e26\u5c07\u5176\u6253\u9020\u6210\u4e86\u7576\u5730\u6700\u5177\u4eba\u6c23\u7684\u6587\u5b78\u66f8\u5e97\uff0c\u7e7c\u7e8c\u50b3\u627f\u8457\u66f8\u4e2d\u7684\u611b\u8207\u77e5\u8b58\uff08DC cat_story_2.txt\uff09\u3002\u5979\u9084\u990a\u4e86\u4e00\u96bb\u9ed1\u8c93\uff0c\u5

## 文檔管理

### POST /documents/text
直接將文本插入 RAG 系統。

In [1]:
# 測試問題
!curl -X POST "http://lightrag:9621/query" \
    -H "Content-Type: application/json" \
    -d '{"query": "什麼是細胞？", "mode": "global"}'

{"response":"Sorry, I'm not able to provide an answer to that question.[no-context]"}

In [2]:
!curl -X POST "http://lightrag:9621/documents/text" \
    -H "Content-Type: application/json" \
    -d '{"text": "細胞是所有生物體的基本單位，所有生物體的結構和功能都離不開細胞。根據細胞的結構，可以將其分為兩類：原核細胞和真核細胞。原核細胞如細菌和藍綠藻，沒有明確的細胞核，遺傳物質直接位於細胞質中。而真核細胞則具有明確的細胞核，並且擁有分工更細的細胞器，如線粒體、內質網等。每個細胞內都包含一個基因組，負責遺傳訊息的儲存與傳遞。細胞的生命活動依賴於複雜的化學反應，這些反應在細胞內的各種細胞器中進行。"}' # description= option description

{"status":"success","message":"Text successfully received. Processing will continue in background."}

伺服器訊息：  
```
Processing 1 document(s)
Extracting stage 1/1: None
Processing d-id: doc-7da4699ea1b8f20db041f46e7f2b7cdd
Chunk 1 of 1 extracted 11 Ent + 14 Rel
Merging stage 1/1: None
Processing: 11 entities and 10 relations (async: 8)
Merge E: 真核細胞 - 線粒體 | 2+0
Merge E: 內質網 - 真核細胞 | 2+0
Completed processing file 1/1: None
Document processing pipeline completed
```

In [3]:
# 測試問題
!curl -X POST "http://lightrag:9621/query" \
    -H "Content-Type: application/json" \
    -d '{"query": "什麼是細胞？", "mode": "global"}'

{"response":"## 細胞概述\n\n細胞是構成所有生物體的基本單位，所有生物體的結構和功能都離不開細胞。這意味著無論是植物、動物還是微生物，它們的生命活動都依賴於細胞的運作。\n\n## 細胞種類\n\n根據細胞結構的不同，細胞可以分為兩類：原核細胞和真核細胞。\n\n*   **原核細胞:** 如細菌和藍綠藻，結構相對簡單，沒有明確的細胞核，遺傳物質直接位於細胞質中。\n*   **真核細胞:** 結構複雜，具有明確的細胞核和各種細胞器，如線粒體、內質網等，構成植物、動物和真菌等高等生物體。\n\n## 細胞組成與功能\n\n細胞內包含一個基因組，負責遺傳訊息的儲存與傳遞。細胞的生命活動依賴於複雜的化學反應，這些反應在細胞內的各種細胞器中進行。細胞內也包含蛋白質、脂類、RNA等重要的生物大分子，共同維持細胞的正常運作。\n\n## 參考文獻\n\n*   [KG] (無檔案路徑)\n*   [DC] null\n*   [KG] (無檔案路徑)\n*   [KG] (無檔案路徑)\n*   [KG] (無檔案路徑)"}

In [6]:
# 測試問題
!curl -X POST "http://lightrag:9621/query" \
    -H "Content-Type: application/json" \
    -d '{"query": "細菌的定義是什麼？", "mode": "global"}'

{"response":"## 細菌的定義\n\n細菌是單細胞微生物，屬於原核細胞。原核細胞是一種結構相對簡單的細胞類型，沒有明確的細胞核，遺傳物質位於細胞質中。細菌廣泛存在於自然界中，是原核細胞的代表性生物之一，另一種代表性生物是藍綠藻。\n\n細菌的生命活動依賴於複雜的化學反應，這些反應在細胞內的各種細胞器中進行。細胞是構成生物體的結構和功能的基本單位，所有生物體的生命活動都離不開細胞。\n\nReferences:\n[KG] \n[KG] \n[KG] \n[KG] \n[KG]"}

載入文件失敗 500 Internal Server Error {"detail":"1 validation error for DocStatusResponse\nfile_path\n Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]\n For further information visit https://errors.pydantic.dev/2.11/v/string_type"} /documents

In [9]:
!curl -X POST "http://lightrag:9621/query" \
    -H "Content-Type: application/json" \
    -d '{"query": "林致遠最後的心願是什麼？他是否如願？", "mode": "hybrid"}'

{"response":"## 林致遠的最後心願\n\n林致遠在生命最後期，最大的願望是能看到愛與知識的傳承。他希望有人能接手知一書屋，延續他對書籍和文學的热爱。在與蘇曉的交往中，他逐漸將書店託付給蘇曉，並希望她能將愛與知識傳承下去 (DC: cat_story_2.txt)。\n\n從故事的結尾來看，林致遠的心願得以實現。蘇曉接手了知一書屋，並將其經營成當地最具人氣的文學書店 (DC: cat_story_2.txt)。她不僅守護了這間書店，更將愛與知識傳遞給更多人。因此，林致遠不僅如願以償，也見證了愛與知識的永恆傳承。\n\n此外，林致遠也希望書店能繼續充滿生氣。在他生命的最後時期，蘇曉的出現為他帶來了慰藉與希望，使他重新燃起了對生活的熱情。書店也因此重煥生機，這也算得上是林致遠心願的實現 (DC: cat_story_2.txt)。\n\n## References\n\n* [KG] cat_story_2.txt\n* [DC] cat_story_2.txt\n* [DC] cat_story_1.txt\n* [KG] cat_story_1.txt\n* [KG] cat_story_2.txt"}

### POST /documents/file
上傳單個文件。

In [13]:
!curl -X POST "http://lightrag:9621/documents/file" \
    -F "file=@./../prince_docs/little_prince_1.txt" \
    #-F "description=" # option description

{"detail":"Not Found"}

### POST /documents/batch
上傳多個文件。

In [ ]:
!curl -X POST "http://lightrag:9621/documents/file" \
    -F "file=@./../prince_docs/little_prince_1.txt" \
    -F "file=@./../prince_docs/little_prince_2.txt" \

### POST /documents/scan
觸發輸入目錄中新文件的文檔掃描。

In [14]:
!curl -X POST "http://lightrag:9621/documents/scan" --max-time 1800 # 根據所有文件的預計索引時間調整 max-time

{"status":"scanning_started","message":"Scanning process has been initiated in the background"}

### DELETE /documents
清除所有文件。

In [14]:
!curl -X DELETE "http://lightrag:9621/documents"

{"status":"success","message":"All documents cleared successfully. Deleted 0 files."}

## 

In [ ]:
!curl http://localhost:9621/api/version

## graph

In [13]:
!curl -X 'GET' \
  'http://lightrag:9621/graphs' \
  -H 'accept: application/json'\
  -d '{"label": "小王子"}' # description= option description


{"detail":[{"type":"missing","loc":["query","label"],"msg":"Field required","input":null}]}

## 實用工具端點

### GET /health
檢查服務器健康狀態和配置。

In [12]:
!curl "http://lightrag:9621/health"

{"status":"healthy","working_directory":"/app/data/rag_storage","input_directory":"/app/data/inputs","configuration":{"llm_binding":"ollama","llm_binding_host":"http://dandelion-ollama-1:11434","llm_model":"gemma3:27b-max-content","embedding_binding":"ollama","embedding_binding_host":"http://dandelion-ollama-1:11434","embedding_model":"bge-m3:latest","max_tokens":32000,"kv_storage":"JsonKVStorage","doc_status_storage":"JsonDocStatusStorage","graph_storage":"NetworkXStorage","vector_storage":"NanoVectorDBStorage","enable_llm_cache_for_extract":true,"enable_llm_cache":true,"workspace":"","max_graph_nodes":1000,"enable_rerank":false,"rerank_model":null,"rerank_binding_host":null},"auth_mode":"disabled","pipeline_busy":true,"keyed_locks":{"process_id":1,"cleanup_performed":{"mp_cleaned":0,"async_cleaned":0},"current_status":{"total_mp_locks":0,"pending_mp_cleanup":0,"total_async_locks":29,"pending_async_cleanup":29}},"core_version":"1.4.4","api_version":"0189","webui_title":null,"webui_des

In [2]:
!apt-get update

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1853 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]  
Get:5 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [48.5 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3207 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5103 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages 

In [3]:
!apt-get install curl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libbrotli1 libcurl4 libnghttp2-14 libpsl5 librtmp1 libssh-4 publicsuffix
The following NEW packages will be installed:
  curl libbrotli1 libcurl4 libnghttp2-14 libpsl5 librtmp1 libssh-4
  publicsuffix
0 upgraded, 8 newly installed, 0 to remove and 78 not upgraded.
Need to get 1308 kB of archives.
After this operation, 3356 kB of additional disk space will be used.
Do you want to continue? [Y/n] ^C
